In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
today = datetime.today().strftime("%d%m%Y")
path = rf'C:\Users\muhammadadlan\Downloads\customer_ultimate_beta\transformed_data\{today}'
daily = rf'C:\Users\muhammadadlan\Downloads\customer_ultimate_beta\daily_report'
report = rf'C:\Users\muhammadadlan\Downloads\customer_ultimate_beta\reports'

In [ ]:
daily_report = rf'{daily}\{today}'

os.makedirs(daily_report, exist_ok=True)

print(f"Directory '{daily_report}' created.")

In [ ]:
reports = rf'{report}\{today}'

os.makedirs(reports, exist_ok=True)

print(f"Directory '{reports}' created.")

In [ ]:
#IMPORT CSVS
customer_df = pd.read_csv(rf'{path}\customer_v2.csv')
sa_df = pd.read_csv(rf'{path}\sa_v2.csv')
sp_df = pd.read_csv(rf'{path}\sp_v2.csv')
payments_df = pd.read_csv(rf'{path}\payments_v2.csv')
ftv_df = pd.read_csv(rf'{path}\ftv_v2.csv')
reversal_df = pd.read_csv(rf'{path}\reversal_v2.csv')

In [ ]:
import pandas as pd

# Read the CSV data
csv_data = rf'{path}\customer_v2.csv'
df = pd.read_csv(csv_data, dtype={'nric':str, 'id':str})

# Group by NRIC and count the unique CICs
result = df.groupby('nric')['id'].nunique().reset_index()

# Rename columns for better understanding
result.columns = ['NRIC', 'Number of unique CICs']

# Display the result
print(result)

# Export the result to a CSV file
output_file = rf'{reports}\unique_cic.csv'
result.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

In [ ]:
# Count the number of unique ids
num_unique_ids = customer_df['id'].nunique()

print(f'There are {num_unique_ids} unique ids in the file.')

In [ ]:
check_cic = customer_df[(customer_df['lastname'] == 'GEOK HUI ANNA TAN')]

print(check_cic['id'])

In [ ]:
import pandas as pd

merged_data = (
    customer_df.merge(ftv_df, left_on='id', right_on='customerId', how='outer')
    .merge(reversal_df, left_on='id', right_on='customerId', how='outer')
    .merge(payments_df, left_on='transactionId_x', right_on='transactionId', how='outer')
    .merge(sa_df, left_on='id', right_on='customerId', suffixes=('', '_savings_account'), how='outer')
    .merge(sp_df, left_on='id', right_on='customerId', suffixes=('', '_savings_pot'), how='outer')
)

In [ ]:
merged_data.to_csv(rf'{reports}\merged_check.csv')

In [ ]:
check_cic = merged_data[(merged_data['lastname'] == 'MUHAMMAD ADLAN HANIF BIN ELIAS')]

print(check_cic['id'])

In [ ]:
# Count the number of unique ids
num_unique_ids = merged_data['id'].nunique()

print(f'There are {num_unique_ids} unique ids in the file.')

In [ ]:
import pandas as pd

# Assuming reports is already defined somewhere in your code
merge = pd.read_csv(rf'{reports}\merged_check.csv')

# Sort the merged DataFrame
merge = merge.sort_values(by=['nric', 'lastname', 'version', 'lastUpdate', 'id'], 
                          ascending=[True, True, True, True, True])

# Drop duplicates based on the 'nric' and 'lastname' columns, keeping the most recent row
merge = merge.drop_duplicates(subset=['nric', 'lastname'], keep='last')

merge.to_csv(rf'{reports}\merged_check_v2.csv')


In [ ]:
check_cic = merge[(merge['lastname'] == 'MUHAMMAD ADLAN HANIF BIN ELIAS')]

print(check_cic['id'])

In [ ]:
remove_unnamed = pd.read_csv(rf'{reports}\merged_check_v2.csv', dtype={'nric':str, 'id':str, 'accounts_0_accountNumber':str})

# remove unnamed:0 columns
remove_unnamed = remove_unnamed.loc[:, ~remove_unnamed.columns.str.contains('^Unnamed')]

# Save the updated data to a new CSV file
remove_unnamed.to_csv(rf'{reports}\merged_data_updated_v2.csv', index=False)

In [ ]:
# Count the number of unique ids
num_unique_ids = remove_unnamed['id'].nunique()

print(f'There are {num_unique_ids} unique ids in the file.')

In [ ]:
import pandas as pd

# Read the CSV file into a DataFrame
ops = pd.read_csv(rf'{reports}\merged_data_updated_v2.csv', dtype={'nric': str, 'id': str, 'accounts_0_accountNumber': str})

# Convert the columns to datetime format and convert the timestamps from UTC to Kuala Lumpur Time (GMT+8)
# Add the names of the columns that should be datetime to this list
datetime_columns = ['createdOn', 'lastUpdate', 'openingTimestamp', 'offboardingStartDate', 'offboardingEndDate', 'tmCreatedAt_x', 'createdAt_x', 'updatedAt_x', 'tmCreatedAt_y', 'createdAt_y', 'updatedAt_y', 'activationTimestamp', 'accounts_0_createdOn', 'accounts_0_lastUpdated', 'preferences_createdOn', 'preferences_lastUpdated', 'devices_0_lastUpdate', 'devices_0_createdOn']

# Convert each datetime column to the proper format and timezone
for column in datetime_columns:
    ops[column] = pd.to_datetime(ops[column], errors='coerce').dt.tz_convert('Asia/Kuala_Lumpur')

# The conditions for an approved customer
is_approved = ((ops['verificationStatus'] == 'APPROVED') & (ops['openingTimestamp'].notnull()) & (ops['paymentStatus_x'] == 'COMPLETED'))

# Customers who have a reversal
has_reversal = ops['tmCreatedAt_y'].notnull()

# Approved customers, regardless of whether a reversal exists or not
approved = ops[is_approved]

# Approved customers who have a reversal
approved_with_reversal = ops[is_approved & has_reversal]

# Concatenate the two DataFrames
approved_combined = pd.concat([approved, approved_with_reversal])

# Drop duplicates based on a unique identifier (I'm using 'id' here, but you should replace it with the appropriate unique identifier for your dataset)
approved_combined = approved_combined.drop_duplicates(subset='id')

# Now, approved_combined contains all unique approved customers, whether they have a reversal or not.

# Drop duplicates based on lastname, keeping the row with the latest openingTimestamp
approved_combined = approved_combined.sort_values('openingTimestamp', ascending=False).drop_duplicates(subset=['id', 'nric'], keep='first')

# Filter out pending or rejected customers
pending_or_rejected = ops[(ops['verificationStatus'] != 'APPROVED') & (~ops['id'].isin(approved_combined['id']))]

# Sort by overallStatus, and openingTimestamp, and drop duplicates based on lastname, keeping the row with the highest priority overallStatus
pending_or_rejected = pending_or_rejected.sort_values(['overallStatus', 'openingTimestamp'], ascending=[False, False])
pending_or_rejected = pending_or_rejected.drop_duplicates('id', keep='first')

# Combine the two filtered DataFrames back together
operation = pd.concat([approved_combined, pending_or_rejected])

# Remove any remaining duplicates
operation = operation.sort_values(['openingTimestamp'], ascending=[False])
operation = operation.drop_duplicates('id', keep='first')

# Group the data by date and calculate the number of daily users for onboard_date
onboard_grouped = operation.groupby(operation['openingTimestamp'].dt.date).agg({'id': 'count'})
onboard_grouped = onboard_grouped.rename(columns={'id': 'Onboarding'})

# Group the data by date and calculate the number of daily users for activation_date
activation_grouped = operation.groupby(operation['tmCreatedAt_x'].dt.date).agg({'accounts_0_accountId': 'count'})
activation_grouped = activation_grouped.rename(columns={'accounts_0_accountId': 'Activation Account'})

# Define the list of offboarding status values
offboarding_statuses = [
    'CLOSURE_AWAITING_CLEARED',
    'CLOSURE_AWAITING_PORTFOLIO_REMOVAL',
    'CLOSURE_AWAITING_SAVING_POT_WITHDRAWAL',
    'CLOSURE_AWAITING_TM_REMOVAL',
    'CLOSURE_DEPOSIT_BLOCK_PROFIT',
    'CLOSURE_TM_REMOVAL'
]

# Filter the DataFrame based on offboarding statuses
offboarding_data = operation[operation['status'].isin(offboarding_statuses)]

# Group the data by date and calculate the number of daily users for offboarding_date
offboarding_grouped = offboarding_data.groupby(offboarding_data['offboardingEndDate'].dt.date).agg({'id': 'count'})
offboarding_grouped = offboarding_grouped.rename(columns={'id': 'Offboarding'})

# Combine the three groups together
result = onboard_grouped.join(activation_grouped, how='outer')
result = result.join(offboarding_grouped, how='outer')

# Fill any missing values (NaN) with 0
result.fillna(0, inplace=True)

# Create a date range with all daily dates
start_date = result.index.min()
end_date = result.index.max()
all_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Reindex the result to include all daily dates
result = result.reindex(all_dates, fill_value=0)
result['Date'] = result.index

# Show only the columns of interest and remove the index
result = result[['Date', 'Onboarding', 'Activation Account', 'Offboarding']]
result.reset_index(drop=True, inplace=True)

# Save the outputs to CSV files
today = pd.to_datetime('today').strftime('%d%m%Y')
operation.to_csv(rf'{daily_report}\customer_modified_{today}.csv', index=False)
result.to_csv(rf'{daily_report}\Daily_User_Onboard.csv', index=False)

result


In [ ]:
customer_modified = pd.read_csv(rf'{daily_report}\customer_modified_{today}.csv', dtype={'nric':str, 'id':str, 'accounts_0_accountNumber':str})

cust_modif = [
    'id', 'createdOn', 'lastUpdate', 'version', 'firstname', 'overallStatus', 'lastname',
    'nickname', 'phoneNumber', 'email', 'nric', 'dateOfBirth','placeOfBirth', 'mailingAddressSameAsResidence',
    'gender', 'nationality', 'type', 'iam', 'preferences', 'devices_0_id', 'devices_0_os', 'devices_0_model',
    'devices_0_installationId', 'devices_0_status', 'devices_0_lastUpdate', 'devices_0_createdOn', 'preferences_id',
    'preferences_pushNotificationsAllowed', 'preferences_marketingEmailFlag', 'preferences_marketingPhoneCallFlag',
    'preferences_marketingSmsFlag', 'preferences_marketingPushFlag', 'preferences_createdOn', 'preferences_lastUpdated',
    'accounts_0_id', 'accounts_0_accountId', 'accounts_0_accountNumber', 'accounts_0_status', 'accounts_0_createdOn',
    'accounts_0_lastUpdated', 'iam_id', 'iam_customer_id', 'iam_username', 'iam_secureword', 'iam_pingOneId',
    'accounts_1_id', 'accounts_1_accountId', 'accounts_1_accountNumber', 'accounts_1_status', 'accounts_1_createdOn',
    'accounts_1_lastUpdated', 'hasRetailsAccount', 'hasSmeAccount', 'offboardingStartDate', 'offboardingEndDate', 'offboardingStatus_string',
    'transactionId_x', 'lifecycleId_x', 'customerAccountId_x', 'customerId_x', 'customerName_x', 'customerAccountType_x', 'customerAccountNumber_x',
    'transferType_x', 'paymentType_x', 'amount_x', 'currency_x', 'paymentStatus_x', 'tmCreatedAt_x', 'createdAt_x', 'updatedAt_x',
    'otherSideFullName_x', 'otherSideExternalId_x', 'otherSideAccountNumber_x', 'otherSideBankCode_x', 'otherSideAccountType_x',
    'recipientReference_x', 'paymentDetails_x', 'proxyRegistrationNumber_x', 'errorCode_x', 'smeId', 'otherSideSmeId',
    'amount_y', 'createdAt_y', 'currency_y', 'customerAccountId_y', 'customerAccountNumber_y', 'customerAccountType_y',
    'customerId_y', 'customerName_y', 'errorCode_y', 'internalRequestId', 'lifecycleId_y', 'otherSideAccountNumber_y', 
    'otherSideAccountType_y', 'otherSideBankCode_y', 'otherSideExternalId_y', 'otherSideFullName_y', 'paymentDetails_y',
    'paymentStatus_y', 'paymentType_y', 'proxyRegistrationNumber_y', 'recipientReference_y', 'tmCreatedAt_y', 'transactionId_y',
    'transferType_y', 'updatedAt_y', 'customerId', 'transactionId', 'verificationStatus', 'reason', 'id_savings_account', 'customerId_savings_account',
    'currency', 'status', 'number', 'openingTimestamp', 'activationTimestamp', 'moneythorAccountKey', 'id_savings_pot', 'customerId_savings_pot',
    'moneythorAccountKey_savings_pot', 'moneythorGoalKey', 'currency_savings_pot', 'category', 'name', 'status_savings_pot', 'imageId', 'openingTimestamp_savings_pot',
    'targetedSavingsGoal','targetedContribution', 'frequency', 'autoContribution', 'contributionDay'
]

cust_modif = customer_modified[cust_modif]

cust_modif.to_csv(rf'{daily_report}\customer_modified_{today}.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

def convert_to_int_if_possible(x):
    try:
        float_val = float(x)
        # Check if the value is infinity or negative infinity
        if float_val == float('inf') or float_val == -float('inf'):
            return x  # or whatever default value you'd like to return for infinity
        return str(int(float_val))
    except ValueError:
        return x


customer_modified = pd.read_csv(fr'{daily_report}\customer_modified_{today}.csv', dtype={'nric':str, 'id':str, 'accounts_0_accountNumber':str})

cust_modif = [
    'id', 'createdOn', 'lastUpdate', 'version', 'firstname', 'overallStatus', 'lastname',
    'nickname', 'phoneNumber', 'email', 'nric', 'dateOfBirth','placeOfBirth', 'mailingAddressSameAsResidence',
    'gender', 'nationality', 'type', 'iam', 'preferences', 'devices_0_id', 'devices_0_os', 'devices_0_model',
    'devices_0_installationId', 'devices_0_status', 'devices_0_lastUpdate', 'devices_0_createdOn', 'preferences_id',
    'preferences_pushNotificationsAllowed', 'preferences_marketingEmailFlag', 'preferences_marketingPhoneCallFlag',
    'preferences_marketingSmsFlag', 'preferences_marketingPushFlag', 'preferences_createdOn', 'preferences_lastUpdated',
    'accounts_0_id', 'accounts_0_accountId', 'accounts_0_accountNumber', 'accounts_0_status', 'accounts_0_createdOn',
    'accounts_0_lastUpdated', 'iam_id', 'iam_customer_id', 'iam_username', 'iam_secureword', 'iam_pingOneId',
    'accounts_1_id', 'accounts_1_accountId', 'accounts_1_accountNumber', 'accounts_1_status', 'accounts_1_createdOn',
    'accounts_1_lastUpdated', 'hasRetailsAccount', 'hasSmeAccount', 'offboardingStartDate', 'offboardingEndDate', 'offboardingStatus_string',
    'transactionId_x', 'lifecycleId_x', 'customerAccountId_x', 'customerId_x', 'customerName_x', 'customerAccountType_x', 'customerAccountNumber_x',
    'transferType_x', 'paymentType_x', 'amount_x', 'currency_x', 'paymentStatus_x', 'tmCreatedAt_x', 'createdAt_x', 'updatedAt_x',
    'otherSideFullName_x', 'otherSideExternalId_x', 'otherSideAccountNumber_x', 'otherSideBankCode_x', 'otherSideAccountType_x',
    'recipientReference_x', 'paymentDetails_x', 'proxyRegistrationNumber_x', 'errorCode_x', 'smeId', 'otherSideSmeId',
    'amount_y', 'createdAt_y', 'currency_y', 'customerAccountId_y', 'customerAccountNumber_y', 'customerAccountType_y',
    'customerId_y', 'customerName_y', 'errorCode_y', 'internalRequestId', 'lifecycleId_y', 'otherSideAccountNumber_y', 
    'otherSideAccountType_y', 'otherSideBankCode_y', 'otherSideExternalId_y', 'otherSideFullName_y', 'paymentDetails_y',
    'paymentStatus_y', 'paymentType_y', 'proxyRegistrationNumber_y', 'recipientReference_y', 'tmCreatedAt_y', 'transactionId_y',
    'transferType_y', 'updatedAt_y', 'customerId', 'transactionId', 'verificationStatus', 'reason', 'id_savings_account', 'customerId_savings_account',
    'currency', 'status', 'number', 'openingTimestamp', 'activationTimestamp', 'moneythorAccountKey', 'id_savings_pot', 'customerId_savings_pot',
    'moneythorAccountKey_savings_pot', 'moneythorGoalKey', 'currency_savings_pot', 'category', 'name', 'status_savings_pot', 'imageId', 'openingTimestamp_savings_pot',
    'targetedSavingsGoal','targetedContribution', 'frequency', 'autoContribution', 'contributionDay'
]

cust_modif = customer_modified[cust_modif]

# List of columns that should be treated as text in the output
text_columns = ['id', 'nric', 'accounts_0_accountNumber', 'phoneNumber', 'customerId_x', 'customerAccountNumber_x', 'otherSideExternalId_x',
                'otherSideAccountNumber_x', 'customerAccountNumber_y', 'customerId_y', 'otherSideAccountNumber_y', 'customerId', 'customerId_savings_account',
                'number']

for column in text_columns:
    # Only convert the column if it exists in the DataFrame
    if column in cust_modif.columns:
        # Apply the conversion function to each element in the column
        cust_modif[column] = cust_modif[column].fillna(-1).apply(convert_to_int_if_possible)

        # Replace the dummy value with an empty string after converting to string
        cust_modif[column] = cust_modif[column].replace('-1', '')

        # Wrap in ="..."
        cust_modif[column] = '="' + cust_modif[column] + '"'

cust_modif.to_csv(fr'{daily_report}\customer_modified_ops_{today}.csv', index=False)

In [ ]:
print(operation.shape)
print(approved.shape)
print(pending_or_rejected.shape)

In [ ]:
import pandas as pd

event = pd.read_csv(rf'{daily_report}\customer_modified_ops_{today}.csv', dtype={'nric':str, 'id':str, 'accounts_0_accountNumber':str})

# Select the desired columns
selected_columns = [
    'id', 'lastname', 'nickname', 'phoneNumber', 'email', 'nric', 'dateOfBirth', 'placeOfBirth',
    'gender', 'nationality', 'overallStatus', 'accounts_0_status', 'type', 'accounts_0_accountId', 'accounts_0_accountNumber',
    'transactionId_x', 'verificationStatus', 'reason', 'lifecycleId_x', 'transferType_x',
    'paymentType_x', 'amount_x', 'currency_x', 'paymentStatus_x', 'tmCreatedAt_x', 'createdAt_x',
    'updatedAt_x', 'otherSideFullName_x', 'otherSideExternalId_x', 'otherSideAccountNumber_x',
    'otherSideBankCode_x', 'otherSideAccountType_x', 'recipientReference_x', 'paymentDetails_x',
    'amount_y',	'currency_y', 'paymentStatus_y', 'tmCreatedAt_y', 'createdAt_y', 'updatedAt_y',
    'errorCode_y', 'internalRequestId', 'otherSideFullName_y', 'otherSideExternalId_y', 'otherSideAccountNumber_y',
    'otherSideBankCode_y', 'otherSideAccountType_y', 'recipientReference_y', 'paymentDetails_y',
    'openingTimestamp', 'activationTimestamp'
]
selected_event = event[selected_columns]

# Rename the columns
selected_event = selected_event.rename(columns={
    'id': 'customerId',
    'accounts_0_accountId': 'tm_account_id',
    'accounts_0_accountNumber': 'account_number',
    'accounts_0_status':'account_status',
    'transactionId_x': 'transactionId',
    'lifecycleId_x':'ftv_lifecycleId', 
    'transferType_x':'ftv_transferType',
    'paymentType_x':'ftv_paymentType',
    'amount_x': 'ftvAmount',
    'currency_x': 'denominator',
    'tmCreatedAt_x':'ftv_tmCreatedAt', 
    'createdAt_x':'ftv_createdAt',
    'updatedAt_x':'ftv_updatedAt',
    'otherSideFullName_x':'ftvOtherSideFullName', 
    'otherSideExternalId_x':'ftvOtherSideExternalId', 
    'otherSideAccountNumber_x':'ftvOtherSideAccountNumber',
    'otherSideBankCode_x':'ftvOtherSideBankCode', 
    'otherSideAccountType_x':'ftvOtherSideAccountType', 
    'recipientReference_x':'ftv_recipientReference', 
    'paymentDetails_x':'ftv_paymentDetails',
    'amount_y':'reversalAmount',	
    'currency_y':'reversalDenominator', 
    'paymentStatus_y':'reversal_paymentStatus', 
    'tmCreatedAt_y':'reversal_tmCreatedAt', 
    'createdAt_y':'reversal_createdAt', 
    'updatedAt_y':'reversal_updatedAt',
    'errorCode_y':'reversal_errorCode', 
    'internalRequestId':'reversal_internalRequestId', 
    'otherSideFullName_y':'reversal_otherSideFullName', 
    'otherSideExternalId_y':'reversal_otherSideExternalId', 
    'otherSideAccountNumber_y':'reversal_otherSideAccountNumber',
    'otherSideBankCode_y':'reversal_otherSideBankCode', 
    'otherSideAccountType_y':'reversal_otherSideAccountType', 
    'recipientReference_y':'reversal_recipientReference', 
    'paymentDetails_y':'reversal_paymentDetails',
})


# Print the selected and renamed DataFrame
print(selected_event)
selected_event.to_csv(rf'{reports}\ops_campaign.csv')


In [ ]:
import pandas as pd

checkers = pd.read_csv(rf'{reports}\ops_campaign.csv', dtype={'nric':str, 'id':str, 'accounts_0_accountNumber':str})

checkers.to_csv(rf'{daily_report}\ops_campaign_no_truncation_{today}.csv', float_format='%.0f', index=False)
# checkers.to_csv(rf'{daily_report}\ops_campaign_no_truncation_{today}.dat', sep='\t', float_format='%.0f', index=False)

In [ ]:
import pandas as pd
from io import StringIO

# Read the CSV data using pandas
campaign_df = pd.read_csv(rf'{reports}\ops_campaign.csv', parse_dates=['ftv_tmCreatedAt', 'openingTimestamp'], dtype={'customerId':str, 'account_number':str})

# Filter the data according to the conditions
filtered_data = campaign_df[(campaign_df['openingTimestamp'] >= '2023-07-21') &
                     (campaign_df['verificationStatus'] == 'APPROVED')]

# Count tmCreatedAt for each date and display the result
result = filtered_data['ftv_tmCreatedAt'].dt.date.value_counts().reset_index()
result.columns = ['Date', 'Activations']
result.sort_values('Date', inplace=True)
result.reset_index(drop=True, inplace=True)
print(result)

result.to_csv(rf'{daily_report}\campaign_activations.csv')


Customer Modified Balances:

In [ ]:
balance_df = pd.read_csv(rf'{path}\balance_cleaned.csv')
customer_modified_df = pd.read_csv(rf'{daily_report}\customer_modified_{today}.csv')

In [ ]:
merged = (
    customer_modified_df.merge(balance_df, left_on='customerAccountId_x', right_on='tm_account_id', how='left')
)

merged.to_csv(rf'{reports}\customer_with_balance_{today}.csv')

In [ ]:
df = pd.read_csv(rf'{reports}\customer_with_balance_{today}.csv')

selected_columns = [
    'id', 'tm_account_id', 'accounts_0_accountNumber', 'createdOn','lastUpdate','version','firstname','overallStatus','lastname','nickname','phoneNumber','email','nric',
    'dateOfBirth','placeOfBirth','mailingAddressSameAsResidence','gender','nationality','type','iam','preferences','devices_0_id','devices_0_os','devices_0_model',
    'ledger_balance','balance_created_balance_denomination', 'total_credit', 'total_debit','last_transaction_date', 'account_address', 'balance_timestamp',
    'devices_0_installationId','devices_0_status','devices_0_lastUpdate','devices_0_createdOn','preferences_id','preferences_pushNotificationsAllowed',
    'preferences_marketingEmailFlag','preferences_marketingPhoneCallFlag','preferences_marketingSmsFlag','preferences_marketingPushFlag',
    'preferences_createdOn','preferences_lastUpdated','accounts_0_accountId','accounts_0_status','accounts_0_createdOn',
    'accounts_0_lastUpdated','iam_id','iam_customer_id','iam_username','iam_secureword','iam_pingOneId','hasRetailsAccount','hasSmeAccount',
    'offboardingStartDate','offboardingEndDate','offboardingStatus_string','transactionId_x','lifecycleId_x','customerAccountId_x','customerId_x',
    'customerName_x','customerAccountType_x','customerAccountNumber_x','transferType_x','paymentType_x','amount_x','currency_x','paymentStatus_x','tmCreatedAt_x',
    'createdAt_x','updatedAt_x','otherSideFullName_x','otherSideExternalId_x','otherSideAccountNumber_x','otherSideBankCode_x','otherSideAccountType_x',
    'recipientReference_x','paymentDetails_x','proxyRegistrationNumber_x','errorCode_x','smeId','otherSideSmeId','amount_y','createdAt_y','currency_y',
    'customerAccountId_y','customerAccountNumber_y','customerAccountType_y','customerId_y','customerName_y','errorCode_y','internalRequestId','lifecycleId_y',
    'otherSideAccountNumber_y','otherSideAccountType_y','otherSideBankCode_y','otherSideExternalId_y','otherSideFullName_y','paymentDetails_y','paymentStatus_y',
    'paymentType_y','proxyRegistrationNumber_y','recipientReference_y','tmCreatedAt_y','transactionId_y','transferType_y','updatedAt_y','verificationStatus',
    'reason','id_savings_account','customerId_savings_account','currency','status','number','openingTimestamp','activationTimestamp','moneythorAccountKey',
    'id_savings_pot','customerId_savings_pot','moneythorAccountKey_savings_pot','moneythorGoalKey','currency_savings_pot','category','name',
    'status_savings_pot','imageId','openingTimestamp_savings_pot','targetedSavingsGoal','targetedContribution','frequency','autoContribution',
    'contributionDay'
]
selected_event = df[selected_columns]


selected_event = selected_event.rename(columns={
    'id': 'customer_id',
    'createdOn': 'createdOn',
    'lastUpdate': 'lastUpdate',
    'version': 'version',
    'firstname':'firstname',
    'overallStatus': 'overallStatus',
    'lastname': 'lastname',
    'nickname': 'nickname',
    'phoneNumber': 'phoneNumber',
    'email': 'email',
    'nric': 'nric',
    'dateOfBirth': 'dateOfBirth',
    'placeOfBirth': 'placeOfBirth',
    'mailingAddressSameAsResidence': 'mailingAddressSameAsResidence',
    'gender': 'gender',
    'nationality': 'nationality',
    'type': 'type',
    'iam': 'iam',
    'preferences': 'preferences',
    'devices_0_id': 'device_id',
    'devices_0_os': 'device_os',
    'devices_0_model': 'device_model',
    'devices_0_installationId': 'device_installationId',
    'devices_0_status': 'device_status',
    'devices_0_lastUpdate': 'device_lastUpdate',
    'devices_0_createdOn': 'device_createdOn',
    'preferences_id': 'preferences_id',
    'preferences_pushNotificationsAllowed': 'preferences_pushNotificationsAllowed',
    'preferences_marketingEmailFlag': 'preferences_marketingEmailFlag',
    'preferences_marketingPhoneCallFlag': 'preferences_marketingPhoneCallFlag',
    'preferences_marketingSmsFlag': 'preferences_marketingSmsFlag',
    'preferences_marketingPushFlag': 'preferences_marketingPushFlag',
    'preferences_createdOn': 'preferences_createdOn',
    'preferences_lastUpdated': 'preferences_lastUpdated',
    'accounts_0_accountId': 'account_accountId',
    'accounts_0_accountNumber': 'account_accountNumber',
    'accounts_0_status': 'account_status',
    'accounts_0_createdOn': 'account_createdOn',
    'accounts_0_lastUpdated': 'account_lastUpdated',
    'iam_id': 'iam_id',
    'iam_customer_id': 'iam_customer_id',
    'iam_username': 'iam_username',
    'iam_secureword': 'iam_secureword',
    'iam_pingOneId': 'iam_pingOneId',
    'hasRetailsAccount': 'hasRetailsAccount',
    'hasSmeAccount': 'hasSmeAccount',
    'offboardingStartDate': 'offboardingStartDate',
    'offboardingEndDate': 'offboardingEndDate',
    'offboardingStatus_string': 'offboardingStatus_string',
    'transactionId_x': 'ftv_transactionId',
    'lifecycleId_x': 'ftv_lifecycleId',
    'customerAccountId_x': 'ftv_customerAccountId',
    'customerId_x': 'ftv_customerId',
    'customerName_x': 'ftv_customerName',
    'customerAccountType_x': 'ftv_customerAccountType',
    'customerAccountNumber_x': 'ftv_customerAccountNumber',
    'transferType_x': 'ftv_transferType',
    'paymentType_x': 'ftv_paymentType',
    'amount_x': 'ftv_amount',
    'currency_x': 'ftv_currency',
    'paymentStatus_x': 'ftv_paymentStatus',
    'tmCreatedAt_x': 'ftv_tmCreatedAt',
    'createdAt_x': 'ftv_createdAt',
    'updatedAt_x': 'ftv_updatedAt',
    'otherSideFullName_x': 'ftv_otherSideFullName',
    'otherSideExternalId_x': 'ftv_otherSideExternalId',
    'otherSideAccountNumber_x': 'ftv_otherSideAccountNumber',
    'otherSideBankCode_x': 'ftv_otherSideBankCode',
    'otherSideAccountType_x': 'ftv_otherSideAccountType',
    'recipientReference_x': 'ftv_recipientReference',
    'paymentDetails_x': 'ftv_paymentDetails',
    'proxyRegistrationNumber_x': 'ftv_proxyRegistrationNumber',
    'errorCode_x': 'ftv_errorCode',
    'smeId': 'ftv_smeId',
    'otherSideSmeId': 'ftv_otherSideSmeId',
    'amount_y': 'reversal_amount',
    'createdAt_y': 'reversal_createdAt',
    'currency_y': 'reversal_currency',
    'customerAccountId_y': 'reversal_customerAccountId',
    'customerAccountNumber_y': 'reversal_customerAccountNumber',
    'customerAccountType_y': 'reversal_customerAccountType',
    'customerId_y': 'reversal_customerId',
    'customerName_y': 'reversal_customerName',
    'errorCode_y': 'reversal_errorCode',
    'internalRequestId_y': 'reversal_internalRequestId',
    'lifecycleId_y': 'reversal_lifecycleId',
    'otherSideAccountNumber_y': 'reversal_otherSideAccountNumber',
    'otherSideAccountType_y': 'reversal_otherSideAccountType',
    'otherSideBankCode_y': 'reversal_otherSideBankCode',
    'otherSideExternalId_y': 'reversal_otherSideExternalId',
    'otherSideFullName_y': 'reversal_otherSideFullName',
    'paymentDetails_y': 'reversal_paymentDetails',
    'paymentStatus_y': 'reversal_paymentStatus',
    'paymentType_y': 'reversal_paymentType',
    'proxyRegistrationNumber_y': 'reversal_proxyRegistrationNumber',
    'recipientReference_y': 'reversal_recipientReference',
    'tmCreatedAt_y': 'reversal_tmCreatedAt',
    'transactionId_y': 'reversal_transactionId',
    'transferType_y': 'reversal_transferType',
    'updatedAt_y': 'reversal_updatedAt',
    'verificationStatus': 'verificationStatus',
    'reason': 'reason',
    'id_savings_account': 'savings_account_id',
    'customerId_savings_account': 'savings_account_cic',
    'currency': 'currency',
    'status': 'status',
    'number': 'number',
    'openingTimestamp': 'openingTimestamp',
    'activationTimestamp': 'activationTimestamp',
    'moneythorAccountKey': 'moneythorAccountKey',
    'id_savings_pot': 'savings_pot_id',
    'customerId_savings_pot': 'savings_pot_cic',
    'moneythorAccountKey_savings_pot': 'savings_pot_moneythorAccountKey',
    'moneythorGoalKey': 'savings_pot_moneythorGoalKey',
    'currency_savings_pot': 'savings_pot_currency',
    'category': 'savings_pot_category',
    'name': 'savings_pot_name',
    'status_savings_pot': 'savings_pot_status',
    'imageId': 'savings_pot_imageid',
    'openingTimestamp_savings_pot': 'savings_pot_openingTimestamp',
    'targetedSavingsGoal': 'savings_pot_targetedSavingsGoal',
    'targetedContribution': 'savings_pot_targetedContribution',
    'frequency': 'savings_pot_frequency',
    'autoContribution': 'savings_pot_autoContribution',
    'contributionDay': 'savings_pot_contributionDay'
})


selected_event.to_csv(rf'{daily_report}\customer_modified_balances_{today}.csv')

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(rf'{reports}\customer_with_balance_{today}.csv')

def convert_to_int_if_possible(x):
    try:
        float_val = float(x)
        # Check if the value is infinity or negative infinity
        if float_val == float('inf') or float_val == -float('inf'):
            return x  # or whatever default value you'd like to return for infinity
        return str(int(float_val))
    except ValueError:
        return x

    
selected_columns = [
    'id', 'tm_account_id', 'accounts_0_accountNumber', 'createdOn','lastUpdate','version','firstname','overallStatus','lastname','nickname','phoneNumber','email','nric',
    'dateOfBirth','placeOfBirth','mailingAddressSameAsResidence','gender','nationality','type','iam','preferences','devices_0_id','devices_0_os','devices_0_model',
    'ledger_balance','balance_created_balance_denomination', 'total_credit', 'total_debit','last_transaction_date', 'account_address', 'balance_timestamp',
    'devices_0_installationId','devices_0_status','devices_0_lastUpdate','devices_0_createdOn','preferences_id','preferences_pushNotificationsAllowed',
    'preferences_marketingEmailFlag','preferences_marketingPhoneCallFlag','preferences_marketingSmsFlag','preferences_marketingPushFlag',
    'preferences_createdOn','preferences_lastUpdated','accounts_0_accountId','accounts_0_status','accounts_0_createdOn',
    'accounts_0_lastUpdated','iam_id','iam_customer_id','iam_username','iam_secureword','iam_pingOneId','hasRetailsAccount','hasSmeAccount',
    'offboardingStartDate','offboardingEndDate','offboardingStatus_string','transactionId_x','lifecycleId_x','customerAccountId_x','customerId_x',
    'customerName_x','customerAccountType_x','customerAccountNumber_x','transferType_x','paymentType_x','amount_x','currency_x','paymentStatus_x','tmCreatedAt_x',
    'createdAt_x','updatedAt_x','otherSideFullName_x','otherSideExternalId_x','otherSideAccountNumber_x','otherSideBankCode_x','otherSideAccountType_x',
    'recipientReference_x','paymentDetails_x','proxyRegistrationNumber_x','errorCode_x','smeId','otherSideSmeId','amount_y','createdAt_y','currency_y',
    'customerAccountId_y','customerAccountNumber_y','customerAccountType_y','customerId_y','customerName_y','errorCode_y','internalRequestId','lifecycleId_y',
    'otherSideAccountNumber_y','otherSideAccountType_y','otherSideBankCode_y','otherSideExternalId_y','otherSideFullName_y','paymentDetails_y','paymentStatus_y',
    'paymentType_y','proxyRegistrationNumber_y','recipientReference_y','tmCreatedAt_y','transactionId_y','transferType_y','updatedAt_y','verificationStatus',
    'reason','id_savings_account','customerId_savings_account','currency','status','number','openingTimestamp','activationTimestamp','moneythorAccountKey',
    'id_savings_pot','customerId_savings_pot','moneythorAccountKey_savings_pot','moneythorGoalKey','currency_savings_pot','category','name',
    'status_savings_pot','imageId','openingTimestamp_savings_pot','targetedSavingsGoal','targetedContribution','frequency','autoContribution',
    'contributionDay'
]

selected_event = df[selected_columns]


selected_event = selected_event.rename(columns={
    'id': 'customer_id',
    'createdOn': 'createdOn',
    'lastUpdate': 'lastUpdate',
    'version': 'version',
    'firstname':'firstname',
    'overallStatus': 'overallStatus',
    'lastname': 'lastname',
    'nickname': 'nickname',
    'phoneNumber': 'phoneNumber',
    'email': 'email',
    'nric': 'nric',
    'dateOfBirth': 'dateOfBirth',
    'placeOfBirth': 'placeOfBirth',
    'mailingAddressSameAsResidence': 'mailingAddressSameAsResidence',
    'gender': 'gender',
    'nationality': 'nationality',
    'type': 'type',
    'iam': 'iam',
    'preferences': 'preferences',
    'devices_0_id': 'device_id',
    'devices_0_os': 'device_os',
    'devices_0_model': 'device_model',
    'devices_0_installationId': 'device_installationId',
    'devices_0_status': 'device_status',
    'devices_0_lastUpdate': 'device_lastUpdate',
    'devices_0_createdOn': 'device_createdOn',
    'preferences_id': 'preferences_id',
    'preferences_pushNotificationsAllowed': 'preferences_pushNotificationsAllowed',
    'preferences_marketingEmailFlag': 'preferences_marketingEmailFlag',
    'preferences_marketingPhoneCallFlag': 'preferences_marketingPhoneCallFlag',
    'preferences_marketingSmsFlag': 'preferences_marketingSmsFlag',
    'preferences_marketingPushFlag': 'preferences_marketingPushFlag',
    'preferences_createdOn': 'preferences_createdOn',
    'preferences_lastUpdated': 'preferences_lastUpdated',
    'accounts_0_accountId': 'account_accountId',
    'accounts_0_accountNumber': 'account_accountNumber',
    'accounts_0_status': 'account_status',
    'accounts_0_createdOn': 'account_createdOn',
    'accounts_0_lastUpdated': 'account_lastUpdated',
    'iam_id': 'iam_id',
    'iam_customer_id': 'iam_customer_id',
    'iam_username': 'iam_username',
    'iam_secureword': 'iam_secureword',
    'iam_pingOneId': 'iam_pingOneId',
    'hasRetailsAccount': 'hasRetailsAccount',
    'hasSmeAccount': 'hasSmeAccount',
    'offboardingStartDate': 'offboardingStartDate',
    'offboardingEndDate': 'offboardingEndDate',
    'offboardingStatus_string': 'offboardingStatus_string',
    'transactionId_x': 'ftv_transactionId',
    'lifecycleId_x': 'ftv_lifecycleId',
    'customerAccountId_x': 'ftv_customerAccountId',
    'customerId_x': 'ftv_customerId',
    'customerName_x': 'ftv_customerName',
    'customerAccountType_x': 'ftv_customerAccountType',
    'customerAccountNumber_x': 'ftv_customerAccountNumber',
    'transferType_x': 'ftv_transferType',
    'paymentType_x': 'ftv_paymentType',
    'amount_x': 'ftv_amount',
    'currency_x': 'ftv_currency',
    'paymentStatus_x': 'ftv_paymentStatus',
    'tmCreatedAt_x': 'ftv_tmCreatedAt',
    'createdAt_x': 'ftv_createdAt',
    'updatedAt_x': 'ftv_updatedAt',
    'otherSideFullName_x': 'ftv_otherSideFullName',
    'otherSideExternalId_x': 'ftv_otherSideExternalId',
    'otherSideAccountNumber_x': 'ftv_otherSideAccountNumber',
    'otherSideBankCode_x': 'ftv_otherSideBankCode',
    'otherSideAccountType_x': 'ftv_otherSideAccountType',
    'recipientReference_x': 'ftv_recipientReference',
    'paymentDetails_x': 'ftv_paymentDetails',
    'proxyRegistrationNumber_x': 'ftv_proxyRegistrationNumber',
    'errorCode_x': 'ftv_errorCode',
    'smeId': 'ftv_smeId',
    'otherSideSmeId': 'ftv_otherSideSmeId',
    'amount_y': 'reversal_amount',
    'createdAt_y': 'reversal_createdAt',
    'currency_y': 'reversal_currency',
    'customerAccountId_y': 'reversal_customerAccountId',
    'customerAccountNumber_y': 'reversal_customerAccountNumber',
    'customerAccountType_y': 'reversal_customerAccountType',
    'customerId_y': 'reversal_customerId',
    'customerName_y': 'reversal_customerName',
    'errorCode_y': 'reversal_errorCode',
    'internalRequestId_y': 'reversal_internalRequestId',
    'lifecycleId_y': 'reversal_lifecycleId',
    'otherSideAccountNumber_y': 'reversal_otherSideAccountNumber',
    'otherSideAccountType_y': 'reversal_otherSideAccountType',
    'otherSideBankCode_y': 'reversal_otherSideBankCode',
    'otherSideExternalId_y': 'reversal_otherSideExternalId',
    'otherSideFullName_y': 'reversal_otherSideFullName',
    'paymentDetails_y': 'reversal_paymentDetails',
    'paymentStatus_y': 'reversal_paymentStatus',
    'paymentType_y': 'reversal_paymentType',
    'proxyRegistrationNumber_y': 'reversal_proxyRegistrationNumber',
    'recipientReference_y': 'reversal_recipientReference',
    'tmCreatedAt_y': 'reversal_tmCreatedAt',
    'transactionId_y': 'reversal_transactionId',
    'transferType_y': 'reversal_transferType',
    'updatedAt_y': 'reversal_updatedAt',
    'verificationStatus': 'verificationStatus',
    'reason': 'reason',
    'id_savings_account': 'savings_account_id',
    'customerId_savings_account': 'savings_account_cic',
    'currency': 'currency',
    'status': 'status',
    'number': 'number',
    'openingTimestamp': 'openingTimestamp',
    'activationTimestamp': 'activationTimestamp',
    'moneythorAccountKey': 'moneythorAccountKey',
    'id_savings_pot': 'savings_pot_id',
    'customerId_savings_pot': 'savings_pot_cic',
    'moneythorAccountKey_savings_pot': 'savings_pot_moneythorAccountKey',
    'moneythorGoalKey': 'savings_pot_moneythorGoalKey',
    'currency_savings_pot': 'savings_pot_currency',
    'category': 'savings_pot_category',
    'name': 'savings_pot_name',
    'status_savings_pot': 'savings_pot_status',
    'imageId': 'savings_pot_imageid',
    'openingTimestamp_savings_pot': 'savings_pot_openingTimestamp',
    'targetedSavingsGoal': 'savings_pot_targetedSavingsGoal',
    'targetedContribution': 'savings_pot_targetedContribution',
    'frequency': 'savings_pot_frequency',
    'autoContribution': 'savings_pot_autoContribution',
    'contributionDay': 'savings_pot_contributionDay'
})

# List of columns that should be treated as text in the output
text_columns = ['customer_id', 'nric', 'account_accountNumber', 'phoneNumber', 'ftv_customerId', 'ftv_customerAccountNumber', 'ftv_otherSideExternalId',
                'ftv_otherSideAccountNumber', 'reversal_customerAccountNumber', 'reversal_customerId', 'reversal_otherSideAccountNumber', 'savings_pot_cic', 'savings_account_cic',
                'number']

for column in text_columns:
    # Only convert the column if it exists in the DataFrame
    if column in selected_event.columns:
        # Apply the conversion function to each element in the column
        selected_event[column] = selected_event[column].fillna(-1).apply(convert_to_int_if_possible)

        # Replace the dummy value with an empty string after converting to string
        selected_event[column] = selected_event[column].replace('-1', '')

        # Wrap in ="..."
        selected_event[column] = '="' + selected_event[column] + '"'
        
selected_event.to_csv(rf'{daily_report}\customer_modified_balances_ops_{today}.csv', index=False)

In [ ]:
df = pd.read_csv(rf'{daily_report}\customer_modified_balances_{today}.csv')

# Count the number of unique ids
num_unique_ids = df['customer_id'].nunique()

print(f'There are {num_unique_ids} unique ids in the file.')

Customer Ultimate Balance

In [ ]:
customer_df = pd.read_csv(rf'{daily_report}\customer_modified_balances_{today}.csv')
employment_df = pd.read_csv(rf'{path}\employment_cleaned.csv')
address_df = pd.read_csv(rf'{path}\address_cleaned.csv')
profile_df = pd.read_csv(rf'{path}\profile_cleaned.csv')

In [ ]:
# rename the variable - and lowercase them
customer_df = customer_df.rename(columns={'devices_0_os': 'device_os'})
customer_df['device_os'] = customer_df['device_os'].str.lower()

customer_df['device_os'].astype('str').str.contains('huawei').sum()

In [ ]:
customer_df['device_os'].astype('category').value_counts()

In [ ]:
# Create a new column 'deviceType' and extract either "iOS", "Android", "Huawei", or "iPadOS"
customer_df['device_os'] = np.where(customer_df['device_os'].str.contains('ios', case=False), 'iOS',
                   np.where(customer_df['device_os'].str.contains('huawei|huawei android', case=False), 'Huawei',
                   np.where(customer_df['device_os'].str.contains('ipad', case=False), 'iPadOS', 
                   'Android')))

In [ ]:
customer_df['device_os'].astype('category').value_counts()

In [ ]:
missing_values = customer_df.isnull().sum()
total_values = customer_df.shape[0]  # Total number of rows in the dataframe

# Calculate the percentage of missing values in each column
missing_percentage = (missing_values / total_values) * 100

# Combine missing values and missing percentage into a single dataframe
missing_data = pd.DataFrame({'Missing Values': missing_values, 'Missing Percentage': missing_percentage})

# Sort the dataframe by the number of missing values in descending order
sorted_missing = missing_data.sort_values(by='Missing Values', ascending=False)

# Print the top 10 columns with the highest number of missing values and their corresponding percentages
print(sorted_missing.head(20))


In [ ]:
customer_df = customer_df.rename(columns={'customer_id': 'customerId', 'lastname': 'fullname'})

columns_to_drop = ['firstname', 'preferences', 'iam', 'proxyRegistrationNumber_x','errorCode_x','proxyRegistrationNumber_y',
                    'otherSideExternalId_y','otherSideSmeId','smeId','iam_secureword','accounts_1_lastUpdated','accounts_1_accountNumber',
                    'accounts_1_status','accounts_1_accountId','accounts_1_id','accounts_1_createdOn','offboardingStartDate',
                    'paymentDetails_y','targetedContribution','offboardingEndDate','reason','customerName_y','otherSideAccountType_y',
                    'internalRequestId','errorCode_y','currency_y']

columns_to_drop_existing = list(set(columns_to_drop).intersection(customer_df.columns))
customer_df = customer_df.drop(columns_to_drop_existing, axis=1)

# remove columns that have "Unnamed" in their column names
customer_df = customer_df.loc[:, ~customer_df.columns.str.contains('^Unnamed')]

In [ ]:
customer_df.to_csv(rf'{reports}\customer_modified_balances_{today}.csv', index=False)

# print the updated size of the dataframe
print(f"Updated size of the dataframe: {customer_df.shape}")

In [ ]:
customer_df.columns

In [ ]:
# Merging
merged_data = (
    customer_df.merge(employment_df, left_on='customerId', right_on='customerId', how='outer')
    .merge(address_df, left_on='customerId', right_on='customerId', how='outer')
    .merge(profile_df, left_on='customerId', right_on='customerId', how='outer')
)

# New DF
merged_data.to_csv(rf'{reports}\merged_balances.csv', index=False)

In [ ]:
df = pd.read_csv(rf'{reports}\merged_balances.csv')
df = df.sort_values(['version'], ascending=[True])

# Drop duplicates based on the customerId' column, keeping the last row - most updated version
print(f"Number of rows before dropping duplicates: {len(df)}")
df = df.drop_duplicates(subset='customerId', keep='last')
print(f"Number of rows after dropping duplicates: {len(df)}")

In [ ]:
print(f"Updated size of the dataframe: {df.shape}")

In [ ]:
# count the number of missing values in each column and their percentage
missing_values = df.isnull().sum()

percent_missing = round((missing_values / len(df)) * 100, 2)
missing_df = pd.concat([missing_values, percent_missing], axis=1, keys=['Missing Count', 'Missing Percentage'])
sorted_missing = missing_df.sort_values(by='Missing Percentage', ascending=False)
print(sorted_missing.head(10))

In [ ]:
# remove columns that have "Unnamed" in their column names
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# df = df.drop(['id_savings_pot', 'targetedSavingsGoal','customerId_savings_pot','currency_savings_pot','status_savings_pot', 
#               'category','frequency','autoContribution','contributionDay', 'moneythorAccountKey'], axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
# Replicate column
subdivision_x_code = df["subdivision_x"].copy()
df["subdivision_x_code"] = subdivision_x_code

# assuming df is your DataFrame with the columns "subdivision", "subdivision_code", and "type_y"
# get the index of the "subdivision" column
subdivision_x_index = df.columns.get_loc("subdivision_x")

# get the index of the "type_y" column
type_x_index = df.columns.get_loc("type_x")

# get the "subdivision_code" column as a Series
subdivision_x_code_column = df.pop("subdivision_x_code")

# insert the "subdivision_code" column after the "subdivision" column and before the "type_y" column
df.insert(subdivision_x_index + 1, "subdivision_x_code", subdivision_x_code_column)

In [ ]:
# Replicate column
subdivision_y_code = df["subdivision_y"].copy()
df["subdivision_y_code"] = subdivision_y_code

# assuming df is your DataFrame with the columns "subdivision", "subdivision_code", and "type_y"
# get the index of the "subdivision" column
subdivision_y_index = df.columns.get_loc("subdivision_y")

# get the index of the "type_y" column
type_y_index = df.columns.get_loc("type_y")

# get the "subdivision_code" column as a Series
subdivision_y_code_column = df.pop("subdivision_y_code")

# insert the "subdivision_code" column after the "subdivision" column and before the "type_y" column
df.insert(subdivision_y_index + 1, "subdivision_y_code", subdivision_y_code_column)


In [ ]:
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("JOHOR", "01")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("KEDAH", "02")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("KELANTAN", "03")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("MELAKA","04")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("NEGERI SEMBILAN", "05")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("PAHANG", "06")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("PULAU PINANG", "07")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("PERAK", "08")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("PERLIS", "09")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("SABAH", "10")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("SARAWAK","11")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("SELANGOR", "12")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("TERENGGANU", "13")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("WPKL","14")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("WP LABUAN","15")
df['subdivision_x_code'] = df['subdivision_x_code'].str.replace("WP PUTRAJAYA","16")


df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("JOHOR", "01")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("KEDAH", "02")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("KELANTAN", "03")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("MELAKA","04")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("NEGERI SEMBILAN", "05")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("PAHANG", "06")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("PULAU PINANG", "07")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("PERAK", "08")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("PERLIS", "09")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("SABAH", "10")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("SARAWAK","11")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("SELANGOR", "12")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("TERENGGANU", "13")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("WPKL","14")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("WP LABUAN","15")
df['subdivision_y_code'] = df['subdivision_y_code'].str.replace("WP PUTRAJAYA","16")

In [ ]:
num_empty_fullname = df['fullname'].isna().sum()
print(f"Number of rows with empty 'fullname': {num_empty_fullname}")

In [ ]:
# Remove rows where 'fullname' is empty
df = df[df['fullname'].notna()]
print(f"Number of rows after removing empty 'fullname': {len(df)}")

In [ ]:
# Convert dateOfBirth to datetime object
# Creating age column
df['dateOfBirth'] = pd.to_datetime(df['dateOfBirth'])

# Calculate age based on current date
currentdate = datetime.today()
df['age'] = (currentdate - df['dateOfBirth']).astype('timedelta64[Y]').astype(int)

# Insert the new "age" column after the "dateOfBirth" column
df.insert(13, 'age', df.pop('age'))

In [ ]:
# Convert ftv_tmCreatedAt column to datetime object
df['ftv_tmCreatedAt'] = pd.to_datetime(df['ftv_tmCreatedAt'], format='%Y-%m-%d %H:%M:%S.%f%z')
# Format ftv_tmCreatedAt column as string with desired format
df['ftv_tmCreatedAt'] = df['ftv_tmCreatedAt'].dt.strftime('%Y-%m-%d %H:%M:%S')
# df = df.sort_values("ftv_tmCreatedAt", ascending=False)

In [ ]:
# # Renaming columns
# df = df.rename(columns={'amount_x': 'ftv_amount', 
#                         'amount_y': 'reversal_amount'})
                        
# Rename the columns
df = df.rename(columns={
                        'type_x_x':'customer_type',
                        'createdOn_y_x': 'timestamp_employment',
                        'line1_x': 'residential_address_line1',
                        'line2_x': 'residential_address_line2',
                        'line3_x': 'residential_address_line3',
                        'line4_x': 'residential_address_line4',
                        'line5_x': 'residential_address_line5',
                        'countryCode_x': 'residential_address_country_code',
                        'city_x': 'residential_address_city',
                        'postalCode_x': 'residential_address_postal_code',
                        'subdivision_x': 'residential_address_subdivision',
                        'subdivision_x_code': 'residential_address_subdivision_code',
                        'type_x': 'residential_address_type',
                        'createdOn_x_y': 'residential_address_createdOn',
                        'lastUpdated_x': 'residential_address_lastUpdated',
                        'smeId_x': 'residential_address_smeId',
                        'line1_y': 'mailing_address_line1',
                        'line2_y': 'mailing_address_line2',
                        'line3_y': 'mailing_address_line3',
                        'line4_y': 'mailing_address_line4',
                        'line5_y': 'mailing_address_line5',
                        'countryCode_y': 'mailing_address_country_code',
                        'city_y': 'mailing_address_city',
                        'postalCode_y': 'mailing_address_postal_code',
                        'subdivision_y': 'mailing_address_subdivision',
                        'subdivision_y_code': 'mailing_address_subdivision_code',
                        'type_y': 'mailing_address_type',
                        'createdOn_y_y': 'mailing_address_createdOn',
                        'lastUpdated_y': 'mailing_address_lastUpdated',
                        'smeId_y': 'mailing_address_smeId',       
                        })

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop(['id_x', 'id_y'], axis=1)

In [ ]:
df.to_csv(rf'{daily_report}\Customer-Ultimate-Balance-{today}.csv', float_format='%.0f', index=False)

Adding FRAML:

In [ ]:
customer_df = pd.read_csv(rf'{daily_report}\Customer-Ultimate-Balance-{today}.csv')
crr_df = pd.read_csv(rf'{path}\crr_cleaned.csv')
complyadv_df = pd.read_csv(rf'{path}\complyadv_cleaned.csv')

In [ ]:
# Merging
merged_data = (
    customer_df.merge(crr_df, left_on='customerId', right_on='CIC', how='outer')
    .merge(complyadv_df, left_on='customerId', right_on='CIC', how='outer')
)

# New DF
merged_data.to_csv(rf'{daily_report}\Customer-Ultimate-Balance-V2-{today}.csv', float_format='%.0f', index=False)

In [ ]:
def convert_to_int_if_possible(x):
    try:
        return str(int(float(x)))
    except ValueError:
        return x
    
# List of columns that should be treated as text in the output
text_columns = ['id', 'nric', 'accounts_0_accountNumber', 'phoneNumber', 'customerId_x', 'customerAccountNumber_x', 'otherSideExternalId_x',
                'otherSideAccountNumber_x', 'customerAccountNumber_y', 'customerId_y', 'otherSideAccountNumber_y', 'customerId', 'customerId_savings_account',
                'number']

for column in text_columns:
    # Only convert the column if it exists in the DataFrame
    if column in merged_data.columns:
        # Apply the conversion function to each element in the column
        merged_data[column] = merged_data[column].fillna(-1).apply(convert_to_int_if_possible)

        # Replace the dummy value with an empty string after converting to string
        merged_data[column] = merged_data[column].replace('-1', '')

        # Wrap in ="..."
        merged_data[column] = '="' + merged_data[column] + '"'
        
merged_data.to_csv(rf'{daily_report}\Customer-Ultimate-Balance-V3-{today}.csv', float_format='%.0f', index=False)

In [ ]:
onboarding_reject_list = pd.read_csv(rf'{daily_report}\Customer-Ultimate-Balance-V2-{today}.csv')

import pandas as pd

# Assuming you have already defined 'today' variable somewhere
# If not, you can define it using: 
crr_time = (pd.Timestamp.now() - pd.Timedelta(days=1)).strftime('%Y-%m-%d')


def convert_to_int_if_possible(x):
    try:
        return str(int(float(x)))
    except ValueError:
        return x

text_columns = [
    'customerId', 'nric', 'accounts_0_accountNumber', 'phoneNumber', 'customerId_x', 'customerAccountNumber_x', 
    'otherSideExternalId_x', 'otherSideAccountNumber_x', 'customerAccountNumber_y', 'customerId_y',
    'otherSideAccountNumber_y', 'customerId', 'customerId_savings_account', 'number'
]

def wrap_with_equals(value):
    if not value.startswith('="'):
        return '="' + value + '"'
    return value

for column in text_columns:
    if column in onboarding_reject_list.columns:
        onboarding_reject_list[column] = onboarding_reject_list[column].fillna(-1).apply(convert_to_int_if_possible)
        onboarding_reject_list[column] = onboarding_reject_list[column].replace('-1', '')
        onboarding_reject_list[column] = onboarding_reject_list[column].apply(wrap_with_equals)

# Define the columns you want to select    
select_cols = ['customerId','fullname', 'overallStatus','phoneNumber', 'email', 
               'device_status', 'device_lastUpdate','account_status', 
               'ftv_paymentStatus', 'verificationStatus','status',
               'ftv_amount', 'ftv_tmCreatedAt','openingTimestamp', 'activationTimestamp','ftv_otherSideFullName', 
               'ftv_otherSideBankCode', 'crr_rating', 'matchStatus', 'nameScreeningMatch', 
               'crr_timestamp']

# Convert the 'crr_timestamp' column to datetime
onboarding_reject_list['crr_timestamp'] = pd.to_datetime(onboarding_reject_list['crr_timestamp'])

# Filter the dataframe for the desired conditions
filtered_df = onboarding_reject_list[
    (onboarding_reject_list['overallStatus'].isin(['ONBOARDED', 'PROSPECT', 'FAILED'])) & 
    (onboarding_reject_list['crr_timestamp'].dt.strftime('%Y-%m-%d') == crr_time)
]

opf = filtered_df[select_cols]


In [ ]:
# Step 1: Rename the columns
rename_map = {
    'customerId': 'CIC NUMBER',
    'overallStatus': 'OVERALL STATUS',
    'fullname': 'NAME',
    'phoneNumber': 'PHONE NUMBER',
    'email': 'EMAIL',
    'device_lastUpdate': 'DEVICE LAST UPDATE',
    'crr_timestamp': 'CRR TIMESTAMP',
    'nameScreeningMatch': 'NAME SCREENING MATCH',
    'matchStatus': 'MATCH STATUS',
    'verificationStatus': 'VERIFICATION STATUS'
}

opf = opf.rename(columns=rename_map)

# Step 2: Reorder columns and add missing columns
desired_columns_order = [
    'CIC NUMBER', 'OVERALL STATUS', 'NAME', 'PHONE NUMBER', 'EMAIL', 
    'DEVICE LAST UPDATE', 'CRR TIMESTAMP', 'NAME SCREENING MATCH', 
    'MATCH STATUS', 'VERIFICATION STATUS', 'REJECT REASON', 'REMARKS', 'SMS1'
]

# For the columns that weren't initially in 'opf', they'll get NaN values
for col in desired_columns_order:
    if col not in opf.columns:
        opf[col] = ''

# Now, reorder the columns
opf = opf[desired_columns_order]

# Save the reordered and renamed DataFrame to a CSV
opf.to_csv(rf'{daily_report}\Onboarded and Failed List {today}.csv', index=False)
